In [3]:
import pandas as pd

In [7]:
retail_data = pd.read_excel('Online Retail.xlsx')


In [9]:
retail_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [13]:
from sklearn.metrics.pairwise import cosine_similarity

In [21]:
# Data cleaning

retail_data.dropna(subset=['CustomerID'], inplace=True)
retail_data = retail_data[retail_data['Quantity'] > 0]

# customer-product matrix
customer_product_matrix = retail_data.pivot_table(index='CustomerID', columns='StockCode', values='Quantity', fill_value=0)

print("\nCustomer-Product Matrix Shape:", customer_product_matrix.shape)

customer_similarity = cosine_similarity(customer_product_matrix)

customer_similarity_df = pd.DataFrame(customer_similarity, index=customer_product_matrix.index, columns=customer_product_matrix.index)

# Recommendation_Function
def recommend_products(customer_id, num_recommendations=5):
    if customer_id not in customer_similarity_df.index:
        return f"Customer ID {customer_id} not found in the dataset."
    
    similar_scores = customer_similarity_df[customer_id]
    
    similar_customers = similar_scores.nlargest(num_recommendations + 1).index[1:]  # Exclude the customer itself

    similar_customers_products = customer_product_matrix.loc[similar_customers]

    recommended_products = similar_customers_products.sum(axis=0)

    recommended_products = recommended_products.sort_values(ascending=False)

    return recommended_products.head(num_recommendations)

try:
    customer_id_input = int(input("Enter Customer ID to get product recommendations: "))
    recommended_products = recommend_products(customer_id_input)

    print(f"\nRecommended products for Customer ID {customer_id_input}:\n{recommended_products}")
except ValueError:
    print("Please enter a valid numeric Customer ID.")


Customer-Product Matrix Shape: (4339, 3665)


Enter Customer ID to get product recommendations:  13047



Recommended products for Customer ID 13047:
StockCode
21899     97.000000
21900     94.000000
21901     91.000000
85123A    58.666667
22986     50.000000
dtype: float64
